In [1]:
!pip install lmfit

  Created wheel for lmfit: filename=lmfit-1.0.1-py3-none-any.whl size=82006 sha256=104e88d9223f17e65ad56481d10f965fcca29694962bf1b6c77a06bdf845bf9f
  Stored in directory: c:\users\fmill\appdata\local\pip\cache\wheels\b6\e0\3d\52a095d9f869ec51e0f8052e4e80bf8e8d844d574e3f7d214b
  Created wheel for asteval: filename=asteval-0.9.21-py3-none-any.whl size=17299 sha256=32d25b7cc16a76536c1af757fa744d83867d45ba824eaeae4c271e60bb1b9f2b
  Stored in directory: c:\users\fmill\appdata\local\pip\cache\wheels\83\4d\3c\06c0b065829c90952351897b82bc35eef1fe5155cdd89dc032
Successfully built lmfit asteval


In [147]:
import sklearn
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


import pandas as pd
import numpy as np

In [142]:
df = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv',
                error_bad_lines=False
                 )

df= df.set_index('Fecha')

In [151]:
data   = iris['data']
labels = iris['target']

In [146]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42, shuffle=True)

In [152]:
u, c = np.unique(y_train, return_counts=True)
print(u,'\n',c)

[0 1 2] 
 [35 39 38]


In [50]:
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']

In [153]:
parameters = {'n_estimators': [100, 200, 400, 1000], 
              'criterion':['entropy', 'gini'],
              'class_weight': ["balanced", "balanced_subsample"]}

In [154]:
rf = RandomForestClassifier()
scoring = ["f1_macro", "recall_macro"]
grid_search = GridSearchCV(rf, parameters, cv=3, scoring=scoring, refit=False)

In [155]:
%%time
grid_search = grid_search.fit(X_train, y_train)

CPU times: user 38.4 s, sys: 191 ms, total: 38.6 s
Wall time: 38.7 s


In [156]:
results_cv = pd.DataFrame(grid_search.cv_results_)
results_cv = results_cv[['params', 'mean_test_f1_macro','std_test_f1_macro']]
results_cv = results_cv.sort_values('mean_test_f1_macro', ascending=False)

In [157]:
results_cv

,params,mean_test_f1_macro,std_test_f1_macro
1,"{'class_weight': 'balanced', 'criterion': 'ent...",0.956042,0.025850
0,"{'class_weight': 'balanced', 'criterion': 'ent...",0.947406,0.022398
3,"{'class_weight': 'balanced', 'criterion': 'ent...",0.947268,0.038259
6,"{'class_weight': 'balanced', 'criterion': 'gin...",0.947268,0.038259
7,"{'class_weight': 'balanced', 'criterion': 'gin...",0.947268,0.038259
10,"{'class_weight': 'balanced_subsample', 'criter...",0.947268,0.038259
11,"{'class_weight': 'balanced_subsample', 'criter...",0.947268,0.038259
13,"{'class_weight': 'balanced_subsample', 'criter...",0.947268,0.038259
14,"{'class_weight': 'balanced_subsample', 'criter...",0.947268,0.038259
15,"{'class_weight': 'balanced_subsample', 'criter...",0.947268,0.038259


In [158]:
results_cv.iloc[0, :]['params']

{'class_weight': 'balanced', 'criterion': 'entropy', 'n_estimators': 200}

Adicionalmente, una vez hayamos definido los hiperparametros podemos hacer

In [160]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
import numpy as np

In [161]:
rf = RandomForestClassifier(n_estimators=200, criterion='entropy', class_weight='balanced')

In [164]:
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

In [169]:
results = cross_validate(rf, X_train, y_train, cv=cv, return_estimator=True, scoring='f1_macro')

In [170]:
results

{'fit_time': array([0.29935932, 0.28405952, 0.28638458, 0.32385039, 0.34898949]),
 'score_time': array([0.0205276 , 0.02039266, 0.01897836, 0.02665019, 0.02451134]),
 'estimator': (RandomForestClassifier(class_weight='balanced', criterion='entropy',
                         n_estimators=200),
  RandomForestClassifier(class_weight='balanced', criterion='entropy',
                         n_estimators=200),
  RandomForestClassifier(class_weight='balanced', criterion='entropy',
                         n_estimators=200),
  RandomForestClassifier(class_weight='balanced', criterion='entropy',
                         n_estimators=200),
  RandomForestClassifier(class_weight='balanced', criterion='entropy',
                         n_estimators=200)),
 'test_score': array([0.91534392, 1.        , 0.93939394, 0.93611111, 0.86425339])}

o tambien podriamos hace un ciclo for e iterar sobre los subset que genera `ShuffleSplit()`

In [174]:
val_f1s = results['test_score']

In [176]:
print('VAL SCORE: {:.2f} +- {:.2f}'.format(np.mean(val_f1s), np.std(val_f1s)))

VAL SCORE: 0.93 +- 0.04


## Final Model
El mejor modelo (osea el que ocuparemos para produccion) puede ser aquel que obtuvo el mejor f1 en el conjunto de test

In [177]:
f1_list = []
for model in results['estimator']:
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')
    f1_list.append(f1)

In [178]:
print('{:.2f} + {:.2f}'.format(np.mean(f1_list), np.std(f1_list)))

0.99 + 0.01


Esta metrica nos permite discriminar entre un modelo u otro

In [ ]:
# 